In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import sys
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

Using TensorFlow backend.


In [2]:
from keras.layers import *
from keras.models import Model, Sequential, load_model
from keras import backend as K
from keras import optimizers
from keras.callbacks import *
from keras.backend import clear_session

In [3]:
df_train = pd.read_csv("X_train.csv")
df_test = pd.read_csv("X_test.csv")
df_y = pd.read_csv("y_train.csv")

In [4]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
y = encoder.fit_transform(df_y["surface"])
y_count = len(list(encoder.classes_))

In [5]:
label_mapping = {i: l for i, l in enumerate(encoder.classes_)}

In [6]:
print(df_train.shape, df_test.shape, y.shape)

num_train = int(df_train.shape[0] / 128)
X_train = np.reshape(np.array(df_train), (num_train,128,13,1))

X_train = X_train[:,:,3:14,:]
num_test = int(df_test.shape[0] / 128)

X_test = np.reshape(np.array(df_test), (num_test,128,13,1))
X_test = X_test[:,:,3:14,:]

y_array = np.array(y)

y_one_hot = np.zeros((y_array.shape[0],y_count))
y_one_hot[np.arange(y_array.shape[0]),y_array] = 1
print(X_train.shape, X_test.shape, y_one_hot.shape)
num_features = X_train.shape[2]

(487680, 13) (488448, 13) (3810,)
(3810, 128, 10, 1) (3816, 128, 10, 1) (3810, 9)


In [7]:
df_train.describe()

,series_id,measurement_number,orientation_X,orientation_Y,orientation_Z,orientation_W,angular_velocity_X,angular_velocity_Y,angular_velocity_Z,linear_acceleration_X,linear_acceleration_Y,linear_acceleration_Z
count,487680.000000,487680.000000,487680.000000,487680.000000,487680.000000,487680.000000,487680.000000,487680.000000,487680.000000,487680.000000,487680.000000,487680.000000
mean,1904.500000,63.500000,-0.018050,0.075062,0.012458,-0.003804,0.000178,0.008338,-0.019184,0.129281,2.886468,-9.364886
std,1099.853353,36.949327,0.685696,0.708226,0.105972,0.104299,0.117764,0.088677,0.229153,1.870600,2.140067,2.845341
min,0.000000,0.000000,-0.989100,-0.989650,-0.162830,-0.156620,-2.371000,-0.927860,-1.268800,-36.067000,-121.490000,-75.386000
25%,952.000000,31.750000,-0.705120,-0.688980,-0.089466,-0.106060,-0.040752,-0.033191,-0.090743,-0.530833,1.957900,-10.193000
50%,1904.500000,63.500000,-0.105960,0.237855,0.031949,-0.018704,0.000084,0.005412,-0.005335,0.124980,2.879600,-9.365300
75%,2857.000000,95.250000,0.651803,0.809550,0.122870,0.097215,0.040527,0.048068,0.064604,0.792263,3.798800,-8.522700
max,3809.000000,127.000000,0.989100,0.988980,0.155710,0.154770,2.282200,1.079100,1.387300,36.797000,73.008000,65.839000


In [8]:
for i in range (num_features):
    print(X_train[:,:,i].mean())

-0.018049738652363475
0.07506199154080025
0.012457660075753735
-0.00380373956336353
0.00017751337388841564
0.008337578268795921
-0.019183612282310716
0.12928096050926643
2.8864677185264584
-9.364885745110374


In [9]:
def model_create():
    
    
    modelb = Sequential()
    

    modelb.add(Dropout(0.1, input_shape=(128,10,1)))
    

    modelb.add(Conv2D(int(60 * 15), (3,1), strides = 1, activation='relu'))
    
   
    modelb.add(Dropout(0.1))
    modelb.add(MaxPooling2D(pool_size = (2,1), strides = (2,1)))
    modelb.add(Conv2D(int(10 * 15), (3,1), strides = 1, activation='relu'))
    modelb.add(Dropout(0.1))
    modelb.add(MaxPooling2D(pool_size = (2,1), strides = (2,1)))
    modelb.add(Conv2D(int(5 * 15), (3,1), strides = 1, activation='relu'))
    modelb.add(Dropout(0.1))
    modelb.add(MaxPooling2D(pool_size = (2,1), strides = (2,1)))
    modelb.add(Conv2D(int(5 * 15), (3,1), strides = 1, activation='relu'))
    modelb.add(Dropout(0.1))
    modelb.add(MaxPooling2D(pool_size = (2,1), strides = (2,1)))
    modelb.add(Conv2D(int(2 * 15), (3,1), strides = 1, activation='relu'))
    modelb.add(Dropout(0.1))
    modelb.add(Conv2D(int(2 * 15), (3,1), strides = 1, activation='relu'))
    

    modelb.add(Flatten())
    
 
    modelb.add(Dropout(0.1))
    modelb.add(Dense(int(2 * 15), activation='relu'))
    modelb.add(Dropout(0.1))
    modelb.add(Dense(int(1 * 15), activation='relu'))
    
  
    modelb.add(Dropout(0.1))
    modelb.add(Dense(y_count, activation = 'softmax'))
    

    modelb.compile(loss='categorical_crossentropy', optimizer='adam')
    return modelb

In [ ]:



folds = KFold(n_splits=15, shuffle=False, random_state=1234)

same = X_train.shape[0]
col = X_train.shape[1]

same_test = X_test.shape[0]
col_test = X_test.shape[1]

prediction = np.zeros((same_test, y_count))
prediction_train = np.zeros((same, y_count))
for fold_n, (train_index, valid_index) in enumerate(folds.split(X_train,y_one_hot)):
    print('Fold', fold_n)
    X_train2, X_valid2 = X_train[train_index], X_train[valid_index]
    y_train2, y_valid2 = y_one_hot[train_index], y_one_hot[valid_index]
    K.clear_session()
    model = model_create()
    print(model.summary())
    checkpointer = ModelCheckpoint('Net1', verbose=1, save_best_only=True)
    earlystopper = EarlyStopping(patience = 15, verbose=0) 
    results = model.fit(X_train2, y_train2, epochs = 300, batch_size = 32,
                    callbacks=[earlystopper, checkpointer], validation_data=[X_valid2, y_valid2])
    model = load_model('Net1')

    prediction_train  += model.predict(X_train)/15
 
    prediction += model.predict(X_test)/15
    print()

In [ ]:
pred_y = prediction_train.argmax(axis = 1)
num_correct = 0
for i in range(y_array.shape[0]):
    if pred_y[i] == y_array[i]:
        num_correct += 1
        
print("Training data score: ", num_correct / y_array.shape[0])

In [43]:
prediction

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [44]:
prediction.shape

(3816, 9)

In [47]:
tS = pd.read_csv('sample_submission.csv')
tS['surface'] = encoder.inverse_transform(prediction.argmax(axis = 1))
tS.to_csv('submission2.csv', index=False)
tS.head(10)

,series_id,surface
0,0,carpet
1,1,carpet
2,2,carpet
3,3,carpet
4,4,carpet
5,5,carpet
6,6,carpet
7,7,carpet
8,8,carpet
9,9,carpet
